# Interfejsy `Parcelable` i `Serializable`

W tym przykładzie będziemy posiadać dwie aktywności, poprzez mechanizm `Intent` włączymy drugą aktywność oraz dzięki zaimplementowaniu interfejsu `Parcelable` oraz `Serializable` przekażemy cały obiekt. Do `activity_main.xml` dodajemy jeden `Button` przez naciśnięcie którego otworzymy drugą aktywność. Wykorzystamy również adnotację `@Parcelize` aby automatycznie wygenerować kod dla implementacji interfejsu `Parcelable`.

`Parcelable` i `Serializable` służą do zrobienia dokładnie tego samego; więc czym się różnią?
- `Parcelable` jest szybszy od `Serializable`
- `Serializable` jest interfejsem znacznikowym, więc implementacja jest szybsza
- `Serializable` tworzy wiele tymczasowych obiektów - jest niewydajny
- możemy przekazać całą tablicę obiektów za pomocą `Parcelable`
- w kotlinie mamy do dyspozycji adnotację `@Parcelize` dzięki której dostajemy domyślną implementację wszystkich metod interfejsu `Parcelable`

Interfejs `Serializable` jest jednym z najprostszych sposobów przenoszenia danych między różnymi częściami aplikacji w Androidzie. Służy do oznaczenia klasy, która może być **zserializowana**, czyli zamieniona na postać, która może być przechowywana lub przesyłana jako ciąg bajtów.

Kiedy klasa implementuje interfejs `Serializable`, można jej obiekty przekazać do innych komponentów aplikacji lub zapisać je na dysku w postaci pliku lub w bazie danych. Przy użyciu `Serializable` można serializować całe drzewa obiektów, w tym ich pola, podklasy itp.

Interfejs `Parcelable` jest alternatywnym sposobem przekazywania obiektów między komponentami w Androidzie. Podobnie jak `Serializable`, służy do zamiany obiektów na postać, która może być przechowywana lub przesyłana między procesami w systemie Android. Jednakże, w porównaniu z `Serializable`, `Parcelable` oferuje lepszą wydajność, szczególnie w przypadku przesyłania dużych obiektów między komponentami.

W przeciwieństwie do `Serializable`, `Parcelable` wymaga ręcznego zaimplementowania metod `writeToParcel()` i `createFromParcel()`, które służą do zamiany obiektu na postać Parcel i odczytanie z niego. `Parcel` to specjalny obiekt, który przechowuje binarną reprezentację danych i jest wykorzystywany do przesyłania danych między komponentami.

Implementacja interfejsu `Parcelable` może być bardziej pracochłonna niż `Serializable`, ale oferuje wiele zalet w kontekście Androida, szczególnie w przypadku przesyłania dużych obiektów między komponentami. Dzięki zastosowaniu `Parcelable` można osiągnąć lepszą wydajność i uniknąć problemów związanych z `Serializable`, takich jak problem z zapętlaniem w przypadku obiektów z referencjami cyklicznymi.

## `Parcelable`

Do `activity_main.xml` dodajemy jeden `Button` przez naciśnięcie którego otworzymy drugą aktywność.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".MainActivity">

    <Button
        android:id="@+id/sendButton"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_margin="16dp"
        android:text="SEND"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

</androidx.constraintlayout.widget.ConstraintLayout>

Następnie tworzymy drugą aktywność `SecondActivity.kt` i uzupełniamy jej layout w pliku `second_activity.xml`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".SecondActivity">

    <TextView
        android:id="@+id/textView"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:textSize="24sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

</androidx.constraintlayout.widget.ConstraintLayout>

Przejdźmy do manifestu i określmy hierarchię aktywności

In [ ]:
        <activity
            android:name=".SecondActivity"
            android:parentActivityName=".MainActivity"
            android:exported="false" />

Następnie utwórzmy klasę której instancję przekażemy pomiędzy aktywnościami.

In [ ]:
class Properties(val a: Int, val b: Int, val c: String?)

Aby być w stanie intancję takiej klasy przekazać do innej aktywności, musi ona implementować interfejs `Parcelable` (lub `Serializable` - więcej o różnicach implementacyjnych poniżej).

In [ ]:
class Properties(val a: Int, val b: Int, val c: String?) : Parcelable {
    constructor(parcel: Parcel) : this(
        parcel.readInt(),
        parcel.readInt(),
        parcel.readString()
    ) {
    }

    override fun writeToParcel(parcel: Parcel, flags: Int) {
        parcel.writeInt(a)
        parcel.writeInt(b)
        parcel.writeString(c)
    }

    override fun describeContents(): Int {
        return hashCode()
    }

    companion object CREATOR : Parcelable.Creator<Properties> {
        override fun createFromParcel(parcel: Parcel): Properties {
            return Properties(parcel)
        }

        override fun newArray(size: Int): Array<Properties?> {
            return arrayOfNulls(size)
        }
    }
}

Jakc widzimy musimy zaimplementować dwie metody, konstruktor oraz **companion object** `Creator`. Ponieważ obiekt towarzyszący jest sam w sobie jest obiektem więc nie ma żadnych ograniczeń - może rozszerzać klasy i implementować interfejsy. Jeżeli chcemy przekazać instancję klasy przez mechanizm `Intent` lub zachować w obiekcie `Bundle` w pierwszym kroku niejawnie zostanie wywołana metoda `writeToParcel`. Metoda ta opakowuje wszystkie właściwości klasy do obiektu `Parcel` wywołując metodę `writeToParcel` z odpowiednim typem. Przy odtwarzaniu instancji klasy `Properties` w pierwszym kroku wywoływana jest metoda `createFromParcel` znajdująca się w obiekcie towarzyszącym `Creator`. Metoda ta wywołuje drugorzędny konstruktor przyjmujący jako argument `Parcel`. Istotnym elementem tej implementacji jest konieczność zachowania kolejności przy przekazywaniu właściwości do i z `Parcel`. Przyjrzyjmy się konstruktorowi drugorzędnemu i metodzie `writeToParcel`.

Innymi słowy metoda `writeToParcel()` zapisuje pola obiektu do obiektu Parcel, a metoda `createFromParcel()` odczytuje te pola z `Parcel`. Następnie, w `Companion object`, definiujemy obiekt `CREATOR` typu `Parcelable.Creator<Properties>`, który jest odpowiedzialny za tworzenie instancji `Properies` z `Parcel`.

In [ ]:
constructor(parcel: Parcel) : this(
    parcel.readInt(),
    parcel.readInt(),
    parcel.readString()
) {
}

override fun writeToParcel(parcel: Parcel, flags: Int) {
    parcel.writeInt(a)
    parcel.writeInt(b)
    parcel.writeString(c)
}

Pola `a`, `b` i `c` są zapisywane do `Parcel` i odczytywane w konstruktorze dokładnie w tej samej kolejności - jest to niezbędne do poprawnego działania - zweróćmy uwagę że nie posługujemy się tutaj żadnym kluczem, czy jakimś unikalnym identyfikartorem.

Ostatnią metodą jest `describeContent` - w interfejsie zdefiniowana jest stała `CONTENT_FILE_DESCRIPTION` która powinna zostać użyta w tej metodzie do zwrócenia maski bitowej - tutaj zwrócimy wynik wwywołania metody `hashCode`.

In [ ]:
override fun describeContents(): Int {
    return hashCode()
}

Przejdźmy do implementacji metody `onClick` przycisku na głównej aktywności. Tym razem zrobimy to nieco inaczej, przechodzimy do pliku `MainActivity.kt`. Do metody `onCreate` dodajemy

In [ ]:
binding.sendButton.setOnClickListener { sendProperties() }

Ustawiamy `onClickListener` przycisku i jako argument przekazywany jest obiekt o typie  interfejsu `OnClickListener` znajdującego się w klasie `View` - implementujemy jako **lambdę**. Klasa `View` jest główną klasą wszystkich elementów interfejsu użytkownika - jest rozszerzana przez wszystkie klasy.

Dodajmy implementację `onClick` w metodzie pomocniczej `sendProperties`

In [ ]:
private fun sendProperties(){
    val intent = Intent(this, SecondActivity::class.java)
        .putExtra(EXTRA_KEY, Properties(1, 2, "String"))
    startActivity(intent)
}

Tworzymy klucz dla `Intent`, następnie tworzymy instancję `Properties` oraz sam `Intent`. Wskazujemy kontekst jako `this` oraz cel którym jest `SecoondActivity::class.java`. Następnie dodajemy za pomocą metody `putExtra` nasz obiekt `Properties` i wywołujemy `startActivity` w celu włączenia drugiej aktywności.

W następnym kroku odbierzemy dane w `SecondActivity` i rozpakujemy obiekt `Properties`. Przejdźmy do pliku `SecondActivity.kt`

In [ ]:
class SecondActivity : AppCompatActivity() {

    private val binding: ActivitySecondBinding by lazy { ActivitySecondBinding.inflate(layoutInflater) }

    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContentView(binding.root)

        val prop =
            if (SDK_INT >= Build.VERSION_CODES.TIRAMISU)
                intent.getParcelableExtra(EXTRA_KEY, Properties::class.java) // min api 33
            else
                intent.getParcelable(EXTRA_KEY) // api 1 - 32

        val text = prop?.a.toString() + prop?.b.toString() + " " + prop?.c + "----\n"
        binding.textView.text = text
    }
}

Tworzymy `Intent` i wywoołujemy metodę `getIntent`, następnie tworzymy obiekt typu `Properties` - tutaj niejawnie zoostanie wywołany konstruktor z argumentem typu `Parcel`. Wyciągamy obiekt za pomocą metody `getParcelableExtra` jako argument podając klucz. Do odpowiednich pól możemy dostać się poprzez odpowiednie **gettery**. Możemy przetestować aplikację.

Od wersji `API 33`, metoda `getParcelableExtra` w klasie `Intent` zwraca instancje klas implementujących interfejs `Parcelable` (lub `Serializable`) z wyłączeniem klasy `Parcelable.Creator` i jego podklas.

Dokładniej mówiąc, od wersji `API 33` metoda `getParcelableExtra` wywołana na obiekcie `Intent` zwróci `null`, jeśli argumentem będzie obiekt klasy `Parcelable.Creator` lub jego podklas. Wcześniej metoda `getParcelableExtra` zwracała instancje tych klas w normalny sposób.

Przykładowo, w poprzednich wersjach Androida, następujący kod działałby poprawnie:

`val myObject = intent.getParcelableExtra<MyParcelableObject>("my_key")`

Jednakże, od wersji `API 33`, powyższy kod zwróciłby `null`, jeśli klasa `MyParcelableObject` dziedziczy od `Parcelable.Creator`.

Nowsze wywołanie wymaga podania typu oczekiwanej wartości, jako drugiego argumentu:

`val myObject = intent.getParcelableExtra(EXTRA_KEY, MyParcelableObject::class.java)`

W tym przypadku, metoda `getParcelableExtra` jest wywoływana z dwoma argumentami:

- `EXTRA_KEY`: klucz, pod którym wartość `MyParcelableObject` została dodana do obiektu `Intent` za pomocą metody `putExtra` w poprzedniej aktywności lub usłudze.
- `Properties::class.java`: typ oczekiwanej wartości, który w tym przypadku jest klasą `MyParcelableObject`.

Przy wywołaniuu metody

```kotlin
intent.getParcelable(EXTRA_KEY)
```

dostaniemy warning związany z przedawnieniem takiego wywołania `getParcelable`. Ponieważ chcemy obsłużyć `API` 28 - 33 (przykładowo), musimy zastosować dwie wersje metody `getParcelable`

In [ ]:
if (SDK_INT >= Build.VERSION_CODES.TIRAMISU)
    intent.getSerializableExtra(EXTRA_SERIALIZABLE, SerializableProperties::class.java) // min api 33
else
    intent.getSerializable(EXTRA_SERIALIZABLE) // api 1 - 32

Aby pozbyć się ostrzeżenia, możemy zdefiniować dodatkową funkcję rozszerzającą, robię to w pliku `StopDepracationWarningUtil`

In [ ]:
inline fun <reified T : Parcelable> Intent.getParcelable(key: String): T? = when {
    Build.VERSION.SDK_INT >= 33 -> getParcelableExtra(key, T::class.java)
    else -> @Suppress("DEPRECATION") getParcelableExtra(key) as? T
}

## **Interfejs `Serializable`**

Dodajmy do projektu kolejną klasę `SerializableProperties`, która implementuje interfejs `Serializable` i powtórzmy wszystkie czynności.

In [ ]:
import java.io.Serializable

class SerializableProperties (val a: Int, b: Int, c: String): Serializable

Interfejs `Serializable` jest **interfejsem znacznikowym** więc nie jest koonieczne implementowanie żadnych metod

W klasie `MainActivity.kt` dodajmy jeszcze jedno pole

In [ ]:
const val EXTRA_SERIALIZABLE = "serializable_example"

Następnie utworzymy instancję klasy `SerializableProperties` i dodamy ją do wcześniej utworzonego `Intent`

In [ ]:
private fun sendProperties(){
    val intent = Intent(this, SecondActivity::class.java)
        .putExtra(EXTRA_KEY, Properties(1, 2, "String"))
        .putExtra(EXTRA_SERIALIZABLE, SerializableProperties(11, 11, "Serializable"))
    startActivity(intent)
}

Przejdźmy do `SecondActivity`, rozpakujmy nasz drugi obiekt i dodajmy jego pola do `TextView`.

In [ ]:
val serializableProperties =
    if (SDK_INT >= Build.VERSION_CODES.TIRAMISU)
        intent.getSerializableExtra(EXTRA_SERIALIZABLE, SerializableProperties::class.java) // min api 33
    else
        intent.getSerializable(EXTRA_SERIALIZABLE) // api 1 - 32

Zwróćmy uwagę że przy wywołaniu metody `getSerializableExtra` dostajemy obiekt o typie interfejsu `Serializable` i potrzebujemy obiekt typu `SerializableProperties`. Ponieważ nasza klasa implementuje ten interfejs, możemy bezpiecznie rzutować na `SerializableProperties`.

Podobnie jak poprzednio, aby aplikacja działała na wersjach Android o `API` 28 - 33, musimy wykorzystać przedawnioną metodę - tutaj również możemy pozbyć się ostrzeżenia przez dodanie odpowiedniej metodyy rozszerzającej.

In [ ]:
inline fun <reified T : java.io.Serializable> Intent.getSerializable(key: String): T? = when {
    Build.VERSION.SDK_INT >= 33 -> getSerializableExtra(key, T::class.java)
    else -> @Suppress("DEPRECATION") getSerializableExtra(key) as? T
}

Po uruchomieniu aplikacji powinniśmy dostać podobny wynik jak poprzednio. Widzimy że właściwie mogę zrobić dokładnie to samo przy pomocy dwóch interfejsów - `Parcelable` i `Serializable`; więc czym się różnią?
- `Parcelable` jest szybszy od `Serializable`
- `Serializable` jest interfejsem znacznikowym, więc implementacja jest szybsza
- `Serializable` tworzy wiele tymczasowych obiektów - jest niewydajny
- możemy przekazać całą tablicę obiektów za pomocą `Parcelable`
- w kotlinie mamy do dyspozycji adnotację `@Parcelize` dzięki której dostajemy domyślną implementację wszystkich metod interfejsu `Parcelable`

## **Adnotacja `@Parcelize`**

Dodajmy kolejną klasę do projektu i jak poprzednio przekażmy jej intstancję jako `Extra` to drugiej aktywności. W pierwszym kroku musimy przejść do pliku `build.gradle(Module)` i dodać plugin

In [ ]:
plugins {
    id 'com.android.application'
    id 'org.jetbrains.kotlin.android'
    id 'kotlin-parcelize'
}

Następnie możemy wykorzystać adnotację `@Parcelize`

In [ ]:
import android.os.Parcelable
import kotlinx.parcelize.Parcelize

@Parcelize
class ParcelizeProperties (val a: Int, val b: Int, val c: String) : Parcelable

Dokonujemy modyfikacji w pliku `MainActivity.kt`

In [ ]:
private fun sendProperties(){
    val intent = Intent(this, SecondActivity::class.java)
        .putExtra(EXTRA_KEY, Properties(1, 2, "String"))
        .putExtra(EXTRA_SERIALIZABLE, SerializableProperties(11, 11, "Serializable"))
        .putExtra(EXTRA_PARCELIZE, ParcelizeProperties(0, 0, "Parcelize"))
    startActivity(intent)
}

Oraz pliku `SecondActivity.kt`

In [ ]:
override fun onCreate(savedInstanceState: Bundle?) {
    super.onCreate(savedInstanceState)
    setContentView(binding.root)

    val prop =
        if (SDK_INT >= Build.VERSION_CODES.TIRAMISU)
            intent.getParcelableExtra(EXTRA_KEY, Properties::class.java) // min api 33
        else
            intent.getParcelable(EXTRA_KEY) // api 1 - 32

    val serializableProperties =
        if (SDK_INT >= Build.VERSION_CODES.TIRAMISU)
            intent.getSerializableExtra(EXTRA_SERIALIZABLE, SerializableProperties::class.java) // min api 33
        else
            intent.getSerializable(EXTRA_SERIALIZABLE) // api 1 - 32


    val parcelizeProperties =
        if (SDK_INT >= Build.VERSION_CODES.TIRAMISU)
            intent.getParcelableExtra(EXTRA_PARCELIZE, ParcelizeProperties::class.java) // min api 33
        else
            intent.getParcelable(EXTRA_PARCELIZE) // api 1 - 32

    val text = prop?.a.toString() + prop?.b.toString() + " " + prop?.c + "----\n" +
        serializableProperties?.a + serializableProperties?.b + " " + serializableProperties?.c + "----\n" +
        parcelizeProperties?.a.toString() + parcelizeProperties?.b.toString() + " " + parcelizeProperties?.c
    binding.textView.text = text
}

Teraz możemy przetestowac aplikację. Widzimy że mamy możliwość automatycznej implementacji wszystkich niezbędnych elementów interfejsu `Parcelable`. Warunkiem jest posiadanie wszystkich pól w **konstruktorze głównym**.